<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-stuff" data-toc-modified-id="Importing-stuff-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing stuff</a></span></li><li><span><a href="#Reading-in-CSV-(from-Veronica)" data-toc-modified-id="Reading-in-CSV-(from-Veronica)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reading in CSV (from Veronica)</a></span></li><li><span><a href="#Cleaning-Up-a-Bit" data-toc-modified-id="Cleaning-Up-a-Bit-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cleaning Up a Bit</a></span></li><li><span><a href="#Get-Patient_Count-Sums-Here" data-toc-modified-id="Get-Patient_Count-Sums-Here-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get Patient_Count Sums Here</a></span></li><li><span><a href="#Making-Unweighted-Plot" data-toc-modified-id="Making-Unweighted-Plot-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Making Unweighted Plot</a></span></li><li><span><a href="#Trying-to-Weight-by-Beds" data-toc-modified-id="Trying-to-Weight-by-Beds-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Trying to Weight by Beds</a></span></li></ul></div>

## Importing stuff

In [182]:
import pandas as pd
import numpy as np
import plotly.express as px

# showing lots of rows and columns
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

# hiding annoying pink warning boxes
import warnings
warnings.filterwarnings("ignore")

## Reading in CSV (from Veronica)

In [183]:
nash_referrals = pd.read_csv('data/nashville_referrals_normalised_only_hospitals_any_avg_day_wait.csv')

## Cleaning Up a Bit

In [184]:
nash_referrals = nash_referrals[nash_referrals['to_npi_specialty'] == 'General Acute Care Hospital']

In [185]:
spec = nash_referrals.groupby('from_npi_specialty')['patient_count'].sum().reset_index()
spec = pd.DataFrame(spec)

In [186]:
spec['percentage'] = (spec['patient_count']/ 1209699) * 100

In [189]:
spec.sort_values('percentage', ascending = False).head(18)

,from_npi_specialty,patient_count,percentage
16,Internal Medicine,435234,35.978702
39,Radiology,361987,29.923725
21,Nurse Practitioner,84351,6.972892
11,Emergency Medicine,74142,6.128963
12,Family Medicine,57359,4.741593
20,"Nurse Anesthetist, Certified Registered",48659,4.022406
29,Pathology,47834,3.954207
2,Anesthesiology,47380,3.916677
26,Orthopaedic Surgery,33886,2.801193
37,Psychiatry & Neurology,28756,2.377120


In [206]:
wanted_spec = ['Internal Medicine','Radiology',
              'Nurse Practitioner','Family Medicine',
              'Pathology',#'Anesthesiology',
              #'Emergency Medicine',#'Nurse Anesthetist, Certified Registered',
              'Orthopaedic Surgery','Surgery','Otolaryngology',
              'Psychiatry & Neurology','Physician Assistant',
              'Specialist','Urology','Ophthalmology']

## Get Patient_Count Sums Here

In [207]:
nash_referrals2 = nash_referrals[['from_npi_specialty','to_facility_name_normalised','patient_count']]
nash_referrals2 = nash_referrals2.groupby(['from_npi_specialty',"to_facility_name_normalised"]).patient_count.sum().reset_index()
nash_referrals2.head()

,from_npi_specialty,to_facility_name_normalised,patient_count
0,Allergy & Immunology,Centennial Medical Center HCA,345
1,Allergy & Immunology,Hendersonville Medical Center HCA,29
2,Allergy & Immunology,Maury Regional Medical Center,29
3,Allergy & Immunology,Summit Medical Center HCA,130
4,Allergy & Immunology,Sumner Regional Medical Center,39


## Making Unweighted Plot

In [208]:
unwanted_hospital = ['Trousdale Medical Center','Saint Thomas Stones River Hospital',
                     'Ashland City Medical Center HCA','Macon County General Hospital',
                    'Riverview Regional Medical Center','Nashville General Hosptial']
unwanted_spec = ['Student in an Organized Health Care Education/Training Program']

nash_referrals2 = nash_referrals2[~nash_referrals2['to_facility_name_normalised'].isin(unwanted_hospital)]
nash_referrals2 = nash_referrals2[nash_referrals2['from_npi_specialty'].isin(wanted_spec)]

In [209]:
nash_referrals['from_npi_specialty'].value_counts(normalize = True).head(20)

Internal Medicine                          0.278599
Nurse Practitioner                         0.115019
Radiology                                  0.112840
Family Medicine                            0.068327
Nurse Anesthetist, Certified Registered    0.060233
Anesthesiology                             0.040778
Emergency Medicine                         0.038911
Orthopaedic Surgery                        0.032062
Psychiatry & Neurology                     0.031440
Surgery                                    0.029105
Physician Assistant                        0.027860
Pathology                                  0.016965
Podiatrist                                 0.014475
Ophthalmology                              0.014008
Specialist                                 0.013852
Urology                                    0.012763
Physical Medicine & Rehabilitation         0.010895
Obstetrics & Gynecology                    0.010584
Hospitalist                                0.010584
Otolaryngolo

## Trying to Weight by Beds

In [210]:
nash_referrals3 = nash_referrals2[['to_facility_name_normalised','patient_count']]
nash_referrals3 = nash_referrals3.groupby(["to_facility_name_normalised"]).patient_count.sum().reset_index()
nash_referrals3.head()

,to_facility_name_normalised,patient_count
0,Centennial Medical Center HCA,124773
1,Hendersonville Medical Center HCA,41106
2,Maury Regional Medical Center,84245
3,NorthCrest Medical Center,23068
4,Saint Thomas Midtown Hospital,59401


In [211]:
nash_referrals3['num_beds'] = nash_referrals3['to_facility_name_normalised']

In [212]:
hosp_beds = {'Centennial Medical Center HCA' : 732,
             'Hendersonville Medical Center HCA' : 129,
             'Maury Regional Medical Center' : 255,
             'Summit Medical Center HCA' : 208,
             'Sumner Regional Medical Center' : 165,
             'Vanderbilt University Lebanon Medical Center' : 245,
             'Vanderbilt University Medical Center' : 1162,
             'Williamson County Hospital' : 203,
             'Saint Thomas Rutherford Hospital' : 322,
             'NorthCrest Medical Center' : 109,
             'Saint Thomas Midtown Hospital' : 396,
             'Saint Thomas West Hospital' : 341,
             'TriStar Horizon Medical Center HCA' : 108,
             'TriStar Skyline Medical Center HCA' : 351,
             'Southern Hills Medical Center HCA' : 117,
             'StoneCrest Medical Center HCA' : 115}

In [213]:
nash_referrals3 = nash_referrals3.replace({"num_beds":hosp_beds})

- make num beds into percent (ex. vandy is 45%)
- 1 - % is weight (vandy's weight is 55%)



In [214]:
nash_referrals3.head(20)

,to_facility_name_normalised,patient_count,num_beds
0,Centennial Medical Center HCA,124773,732
1,Hendersonville Medical Center HCA,41106,129
2,Maury Regional Medical Center,84245,255
3,NorthCrest Medical Center,23068,109
4,Saint Thomas Midtown Hospital,59401,396
5,Saint Thomas Rutherford Hospital,57724,322
6,Saint Thomas West Hospital,107552,341
7,Southern Hills Medical Center HCA,25540,117
8,StoneCrest Medical Center HCA,22328,115
9,Summit Medical Center HCA,54330,208


In [215]:
nash_referrals3['num_beds'].sum()

4958

In [216]:
nash_referrals3['percent_of_total_beds'] = (nash_referrals3['num_beds']/ 4958) * 100

In [221]:
fig2 = px.scatter(nash_referrals3, x='num_beds', y='patient_count',
                  hover_name = 'to_facility_name_normalised',
                  title = "Number of Patient Referrals Vs. Number of Hospital Beds")
fig2.update_yaxes(title="Patient Refferal Count")
fig2.update_xaxes(title="Number of Hospital Beds")
fig2.update_traces(marker=dict(size=10, color = 'DodgerBlue',
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

 #line of best fit y = 255.281657x - 7633.216001
fig2.add_shape(type='line',line_width=2, line_dash="dash",
                x0=0,
                y0=-7633.2,
                x1=1400,
                y1=349761.1,
                line=dict(color='tomato',),
                xref='x',
                yref='y'
)
fig2.show()

In [218]:
fig = px.bar(nash_referrals2, x='patient_count', y='to_facility_name_normalised',
             color = 'from_npi_specialty', title = 'Referrals by Specialty')
fig.update_yaxes(title=None)
fig.update_xaxes(title="Patient Count")
fig.show()

In [219]:
fig.write_html("Referrals_by_Specialty_w_Vandy.html")